<a href="https://colab.research.google.com/github/sangttruong/IncomeVis/blob/master/VizIncIneq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Technical Appendix for Visualizing Income Inequality in the United States**

*Author: [Sang Truong](mailto:sangtruong_2021@depauw.edu) and [Prof. Humberto Barreto](mailto:hbarreto@depauw.edu)*

Department of Economics and Management

DePauw University, Greencastle, IN 46135


## **Section 1. Introduction**

This notebook supports Barreto and Truong's 2020 paper, "Visualizing Income Inequality in the United States." It gives a detailed explanation of the transformation of the raw data to the 3D visualization.

Abstract:
Information about the distribution of income is communicated by tabulation and statistics (e.g., the Gini coefficient). These can be difficult to understand and often require expert knowledge. Our goal is to provide a novel, eye-catching visual display of the income distribution in the United States that conveys fundamental information about the evolution and current level of income inequality to a wide audience. We use IPUMS CPS data to create household income deciles adjusted for price level and household size for each of the 50 states and DC from 1976 to 2018. Plotting these data gives a 3D chart that provides a startling picture of income differences within and across states over time. The visualization is thought provoking and can be used at all levels to stimulate further investigation. 

Before getting started, we import libraries: *pandas* for data manipulation and analysis and *numpy* for scientific computing. We also mount Google Drive to Google Colab so that all of our analyses are conducted online.

In [ ]:
import pandas as pd
import numpy as np
import json

from collections import OrderedDict
from google.colab import drive

drive.mount('/content/gdrive')
in_path = 'gdrive/My Drive/Colab Notebooks/USIncomeVis/input/'
out_path = 'gdrive/My Drive/Colab Notebooks/USIncomeVis/output/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Data for our analysis are from [IPUMS-CPS](https://cps.ipums.org/cps/). We download all available years in one data extract from CPS. Documentation on downloading and variables can be found on the IPUMS website.

**Reference:**

Sarah Flood, Miriam King, Renae Rodgers, Steven Ruggles and J. Robert Warren. Integrated Public Use Microdata Series, Current Population Survey: Version 6.0 [dataset]. Minneapolis, MN: IPUMS, 2018.
https://doi.org/10.18128/D030.V6.0

For IPUMS-CPS, all selected samples are ASEC samples. Although IPUMS CPS has data back to 1962, geographic location (STATEFIP) is only available since 1977:
"STATEFIP is comparable for 1963-1967 and 1977 onward, years in which each state and the District of Columbia were separately identified. In the remaining years, two or more states share the same code, and these groupings change over time. In 1962, 8 states cannot be separately identified. In 1968-1972, 32 states cannot be separately identified, and in 1973-1976, 38 states cannot be separately identified. In these years, up to 5 states share the same code."
**Reference:** https://cps.ipums.org/cps-action/variables/statefip#comparability_section 

We download ASEC samples from 1977 to 2019 as csv in a zip file, extract (~600MB), rename as "ipums-cps.csv" and place it in an accessible Google drive location.

For updating, ASEC comes out in IPUMS in October.

Years are confusing. We have to be careful. We have ASEC CPS from 1977 to 2019. The CPI99 and HHINCOME variables in a sample year is for the previous year. So, the 1977 sample has Consumr Price Index and Household Income for 1976. Thus, we compute RHHINCOME from sample year 1977 data that is actually real household income for 1976.

In [ ]:
# Import raw data
raw = pd.read_csv(in_path + "ipums-cps.csv")
raw = raw[raw.YEAR >= 1977] # This line should be here regardless if you have the data before 1977 or not. It won't hurt if you don't have the data.
raw.head(5)

,YEAR,SERIAL,MONTH,CPSID,ASECFLAG,HFLAG,ASECWTH,CPI99,STATEFIP,HHINCOME,PERNUM,CPSIDP,ASECWT,RELATE,AGE,SEX,RACE,HISPAN,EDUC
135351,1977,2,3,NaN,1,NaN,856.04,2.928,23,11500,1,NaN,856.04,101,48,2,100,0,110
135352,1977,2,3,NaN,1,NaN,856.04,2.928,23,11500,2,NaN,803.60,301,15,1,100,0,32
135353,1977,3,3,NaN,1,NaN,856.04,2.928,23,44310,1,NaN,856.04,101,47,1,100,0,122
135354,1977,3,3,NaN,1,NaN,856.04,2.928,23,44310,2,NaN,856.04,201,45,2,100,0,72
135355,1977,3,3,NaN,1,NaN,856.04,2.928,23,44310,3,NaN,814.43,301,19,2,100,0,80


In [ ]:
raw.describe()

,YEAR,SERIAL,MONTH,CPSID,ASECFLAG,HFLAG,ASECWTH,CPI99,STATEFIP,HHINCOME,PERNUM,CPSIDP,ASECWT,RELATE,AGE,SEX,RACE,HISPAN,EDUC
count,7.538032e+06,7.538032e+06,7538032.0,5.587112e+06,7538032.0,199556.000000,7.538032e+06,7.538032e+06,7.538032e+06,7.538032e+06,7.538032e+06,5.587112e+06,7.538032e+06,7.538032e+06,7.538032e+06,7.538032e+06,7.538032e+06,7.538032e+06,7.538032e+06
mean,1.999107e+03,4.315759e+04,3.0,1.460444e+13,1.0,0.301374,1.558062e+03,1.192234e+00,2.800531e+01,5.830267e+04,2.278207e+00,1.460444e+13,1.580876e+03,2.701338e+02,3.432149e+01,1.516254e+00,1.462156e+02,3.454306e+01,5.894529e+01
std,1.246539e+01,2.589295e+04,0.0,8.902060e+12,0.0,0.458856,1.010589e+03,5.625683e-01,1.573892e+01,6.831768e+04,1.400125e+00,8.902060e+12,1.045842e+03,2.524557e+02,2.212700e+01,4.997358e-01,1.385056e+02,1.126688e+02,3.983980e+01
min,1.977000e+03,1.000000e+00,3.0,0.000000e+00,1.0,0.000000,0.000000e+00,6.630000e-01,1.000000e+00,-3.704000e+04,1.000000e+00,0.000000e+00,0.000000e+00,1.010000e+02,0.000000e+00,1.000000e+00,1.000000e+02,0.000000e+00,1.000000e+00
25%,1.988000e+03,2.111900e+04,3.0,0.000000e+00,1.0,0.000000,8.741400e+02,7.740000e-01,1.300000e+01,2.008000e+04,1.000000e+00,0.000000e+00,8.744900e+02,1.010000e+02,1.500000e+01,1.000000e+00,1.000000e+02,0.000000e+00,2.000000e+01
50%,2.001000e+03,4.199800e+04,3.0,1.997030e+13,1.0,0.000000,1.495020e+03,9.670000e-01,2.900000e+01,4.000000e+04,2.000000e+00,1.997030e+13,1.500770e+03,2.010000e+02,3.300000e+01,2.000000e+00,1.000000e+02,0.000000e+00,7.200000e+01
75%,2.010000e+03,6.314200e+04,3.0,2.008020e+13,1.0,1.000000,1.978940e+03,1.467000e+00,4.100000e+01,7.430000e+04,3.000000e+00,2.008020e+13,2.014180e+03,3.010000e+02,5.100000e+01,2.000000e+00,1.000000e+02,0.000000e+00,8.100000e+01
max,2.019000e+03,9.998600e+04,3.0,2.019031e+13,1.0,1.000000,2.865431e+04,2.928000e+00,5.600000e+01,3.299997e+06,2.600000e+01,2.019031e+13,4.442383e+04,1.260000e+03,9.900000e+01,2.000000e+00,8.300000e+02,9.020000e+02,1.250000e+02


"The Census Bureau fielded the CPS 2014 ASEC sample using an experimental redesign. All respondents received new health insurance questions, but 3/8ths of the total sample was randomly selected to receive the redesigned income questions. The larger portion of the sample (5/8) was given the existing questions on income. The redesign attempted to address income under-reporting, in particular, retirement, pensions, annuities, and government cash-transfer programs. More accurate income reporting in turn allows for better measurement of poverty statistics."

HFLAG = 0 indicates 5/8 sample, and HFLAG = 1 indicate 3/8 sample in 2014. For the 2014 to be compatible with other data year, we select HFLAG = 0.

**Reference:** https://cps.ipums.org/cps/three_eighths.shtml).

In [ ]:
# Select data with HFLAG != 1 and then drop HFLAG
raw = raw[raw.HFLAG !=1]
raw = raw.drop(columns = ['HFLAG'])
raw.head(5)

,YEAR,SERIAL,MONTH,CPSID,ASECFLAG,ASECWTH,CPI99,STATEFIP,HHINCOME,PERNUM,CPSIDP,ASECWT,RELATE,AGE,SEX,RACE,HISPAN,EDUC
135351,1977,2,3,NaN,1,856.04,2.928,23,11500,1,NaN,856.04,101,48,2,100,0,110
135352,1977,2,3,NaN,1,856.04,2.928,23,11500,2,NaN,803.60,301,15,1,100,0,32
135353,1977,3,3,NaN,1,856.04,2.928,23,44310,1,NaN,856.04,101,47,1,100,0,122
135354,1977,3,3,NaN,1,856.04,2.928,23,44310,2,NaN,856.04,201,45,2,100,0,72
135355,1977,3,3,NaN,1,856.04,2.928,23,44310,3,NaN,814.43,301,19,2,100,0,80


We create labels for states and establish starting color scheme for the visualization.

In [ ]:
# States
statefip = list(set(raw.STATEFIP))
state_name = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
              'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
              'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
              'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
              'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',
              'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
              'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
              'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
              'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
              'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

colors = ["#FF0000", "#FF0A00", "#FF1400", "#FF1E00", "#FF2800", "#FF3300", "#FF3D00",
          "#FF4700", "#FF5100", "#FF5B00", "#FF6600", "#FF7000", "#FF7A00", "#FF8400",
          "#FF8E00", "#FF9900", "#FFA300", "#FFAD00", "#FFB700", "#FFC100", "#FFCC00",
          "#FFD600", "#FFE000", "#FFEA00", "#FFF400", "#FFFF00", "#F4FF00", "#EAFF00",
          "#E0FF00", "#D6FF00", "#CCFF00", "#C1FF00", "#B7FF00", "#ADFF00", "#A3FF00",
          "#99FF00", "#8EFF00", "#84FF00", "#7AFF00", "#70FF00", "#66FF00", "#5BFF00",
          "#51FF00", "#47FF00", "#3DFF00", "#32FF00", "#28FF00", "#1EFF00", "#14FF00",
          "#0AFF00", "#00FF00"]
colors = pd.DataFrame(colors, columns = ['Color'], index = statefip)

## **Section 2. Adjusting HHINCOME**

In this analysis, we employ 2 deflators to make better comparisons: 1) Consumer Price Index (CPI) to create real dollar values of household income over time (RHHINCOME) and 2) Household size (HHSIZE) to adjust for the number of people in a household (ERHHINCOME). 

1) Because of inflation, to compare incomes over time, we need to convert nominal dollar values of HHINCOME to real values. We adjust incomes so they are all in 2018 dollars. 

2) We want to compare household incomes, but a household with two people with a total income of $100,000 is better off than a six-person household with the same income. We could just divide by the number of people in the household, but that ignores the fact that some expenses don't scale up linearly (a two-bedroom apartment is not double the rent of a one-bedroom and two people could share a car, for example). An *equivalence scale* adjusts household size. There are several options available (http://www.oecd.org/els/soc/OECD-Note-EquivalenceScales.pdf). We use the OECD scale. It is explained below.

### **2.1. Consumer Price Index**

CPI reported by IPUMS-CPS is based on 1999 dollars. We convert CPI99 to 2018 dollars by multiplying by 1.507. Remember, CPI in a given sample year is actually the CPI for the previous year.
This web page gives more explanation: https://cps.ipums.org/cps/cpi99.shtml

[HHINCOME](https://cps.ipums.org/cps-action/variables/HHINCOME#description_section) reports the total money income during the **previous calendar** year of all adult household members. "The amount should equal the sum of all household members' individual incomes as recorded in the IPUMS-CPS variable [INCTOT](https://cps.ipums.org/cps-action/variables/INCTOT#description_section). The persons included were those present in the household at the time of the survey. People who lived in the household during the previous year but were not still living there at the time of the survey are not included; household members who lived elsewhere during the previous year but had joined the household at the time of the survey are included."

HHINCOME includes sources of income like wages, salaries, and business income. It can be negative. Some of the component income sources (like [INCWAGE](https://cps.ipums.org/cps-action/variables/INCWAGE#description_section) have "disclosure avoidance measures" for individuals with high incomes.

Our income range is from the 5th to the 95th percentile so we avoid negative values and problems with correctly measuring extremely high incomes.

In [ ]:
# Generate Real HHINCOME in 2018 dollars
raw["RHHINCOME"] = np.nan
raw.RHHINCOME = raw.HHINCOME*raw.CPI99*1.507

raw.head(10)

,YEAR,SERIAL,MONTH,CPSID,ASECFLAG,ASECWTH,CPI99,STATEFIP,HHINCOME,PERNUM,CPSIDP,ASECWT,RELATE,AGE,SEX,RACE,HISPAN,EDUC,RHHINCOME
135351,1977,2,3,NaN,1,856.04,2.928,23,11500,1,NaN,856.04,101,48,2,100,0,110,50743.704000
135352,1977,2,3,NaN,1,856.04,2.928,23,11500,2,NaN,803.60,301,15,1,100,0,32,50743.704000
135353,1977,3,3,NaN,1,856.04,2.928,23,44310,1,NaN,856.04,101,47,1,100,0,122,195517.697760
135354,1977,3,3,NaN,1,856.04,2.928,23,44310,2,NaN,856.04,201,45,2,100,0,72,195517.697760
135355,1977,3,3,NaN,1,856.04,2.928,23,44310,3,NaN,814.43,301,19,2,100,0,80,195517.697760
135356,1977,3,3,NaN,1,856.04,2.928,23,44310,4,NaN,773.29,301,16,2,100,0,50,195517.697760
135357,1977,3,3,NaN,1,856.04,2.928,23,44310,5,NaN,846.88,301,12,2,100,0,1,195517.697760
135358,1977,4,3,NaN,1,801.57,2.928,23,23314,1,NaN,801.57,101,63,1,100,902,72,102872.931744
135359,1977,4,3,NaN,1,801.57,2.928,23,23314,2,NaN,801.57,201,63,2,100,0,72,102872.931744
135360,1977,5,3,NaN,1,657.13,2.928,23,8805,1,NaN,657.13,101,29,2,100,0,121,38852.027280


In [ ]:
raw.describe()

,YEAR,SERIAL,MONTH,CPSID,ASECFLAG,ASECWTH,CPI99,STATEFIP,HHINCOME,PERNUM,CPSIDP,ASECWT,RELATE,AGE,SEX,RACE,HISPAN,EDUC,RHHINCOME
count,7.477891e+06,7.477891e+06,7477891.0,5.526971e+06,7477891.0,7.477891e+06,7.477891e+06,7.477891e+06,7.477891e+06,7.477891e+06,5.526971e+06,7.477891e+06,7.477891e+06,7.477891e+06,7.477891e+06,7.477891e+06,7.477891e+06,7.477891e+06,7.477891e+06
mean,1.998987e+03,4.283406e+04,3.0,1.460917e+13,1.0,1.529195e+03,1.196072e+00,2.800799e+01,5.805103e+04,2.278471e+00,1.460917e+13,1.551674e+03,2.700711e+02,3.430549e+01,1.516255e+00,1.460118e+02,3.460769e+01,5.889825e+01,8.583669e+04
std,1.244336e+01,2.573148e+04,0.0,8.898738e+12,0.0,9.112964e+02,5.631891e-01,1.573709e+01,6.795189e+04,1.400422e+00,8.898738e+12,9.445234e+02,2.523812e+02,2.212238e+01,4.997357e-01,1.381461e+02,1.128712e+02,3.982632e+01,8.162448e+04
min,1.977000e+03,1.000000e+00,3.0,0.000000e+00,1.0,0.000000e+00,6.630000e-01,1.000000e+00,-3.704000e+04,1.000000e+00,0.000000e+00,0.000000e+00,1.010000e+02,0.000000e+00,1.000000e+00,1.000000e+02,0.000000e+00,1.000000e+00,-7.860026e+04
25%,1.988000e+03,2.096000e+04,3.0,0.000000e+00,1.0,8.700900e+02,7.740000e-01,1.300000e+01,2.002200e+04,1.000000e+00,0.000000e+00,8.703500e+02,1.010000e+02,1.500000e+01,1.000000e+00,1.000000e+02,0.000000e+00,2.000000e+01,3.694356e+04
50%,2.001000e+03,4.165700e+04,3.0,1.997030e+13,1.0,1.489790e+03,9.670000e-01,2.900000e+01,4.000000e+04,2.000000e+00,1.997030e+13,1.495650e+03,2.010000e+02,3.300000e+01,2.000000e+00,1.000000e+02,0.000000e+00,7.200000e+01,6.826866e+04
75%,2.010000e+03,6.264400e+04,3.0,2.008010e+13,1.0,1.967100e+03,1.467000e+00,4.100000e+01,7.400000e+04,3.000000e+00,2.008010e+13,2.001220e+03,3.010000e+02,5.100000e+01,2.000000e+00,1.000000e+02,0.000000e+00,8.100000e+01,1.104051e+05
max,2.019000e+03,9.998600e+04,3.0,2.019031e+13,1.0,1.795753e+04,2.928000e+00,5.600000e+01,3.299997e+06,2.600000e+01,2.019031e+13,1.998275e+04,1.260000e+03,9.900000e+01,2.000000e+00,8.300000e+02,9.020000e+02,1.250000e+02,3.451328e+06


### **2.2. Household Size**

Since our unit of analysis is household, we initialize a new dataframe (hhsize, or household size) and a new variable (ISIZE, or individual size) to account for the different in household size. For household $j$ in the sample with $n$ members,

$$ HHSIZE_j = \sum_{i = 1}^{n} ISIZE_{n} $$

Each member in the household contributes to the household size depending on age. Specifically:
* Household head (PERNUM = 1): ISIZE = 1 unit.
* Adult (PERNUM $\neq$ 1 and AGE > 16): ISIZE = 0.7 unit.
* Child (PERNUM $\neq$ 1 and AGE $\leq$ 16): ISIZE = 0.5 unit.

As mentioned previously, this is the OECD scale. Here is another reference.

**Reference:**

"The OECD scale assigns a weight of 1.0 to the first adult, 0.7 to each additional adult, and 0.5 to each child. In the PSID, a child is a family member age 17 or younger. In the CPS and the CEX, we define a child as age 16 or younger. The original OECD definition is age 13 or younger."

Footnote 32, page 27, "Unequal We Stand: An Empirical Analysis of Economic Inequality in the United States, 1967—2006"
Jonathan Heathcote, Fabrizio Perri, and Giovanni L. Violante
https://ideas.repec.org/c/red/ccodes/09-214.html  

In [ ]:
# Generate size
raw["ISIZE"] = np.nan

raw.loc[raw["PERNUM"] == 1, "ISIZE"] = 1
raw.loc[(raw["PERNUM"] != 1) & (raw["AGE"] > 16), "ISIZE"] = 0.7
raw.loc[(raw["PERNUM"] != 1) & (raw["AGE"] <= 16), "ISIZE"] = 0.5

raw.head(5)

,YEAR,SERIAL,MONTH,CPSID,ASECFLAG,ASECWTH,CPI99,STATEFIP,HHINCOME,PERNUM,CPSIDP,ASECWT,RELATE,AGE,SEX,RACE,HISPAN,EDUC,RHHINCOME,ISIZE
135351,1977,2,3,NaN,1,856.04,2.928,23,11500,1,NaN,856.04,101,48,2,100,0,110,50743.70400,1.0
135352,1977,2,3,NaN,1,856.04,2.928,23,11500,2,NaN,803.60,301,15,1,100,0,32,50743.70400,0.5
135353,1977,3,3,NaN,1,856.04,2.928,23,44310,1,NaN,856.04,101,47,1,100,0,122,195517.69776,1.0
135354,1977,3,3,NaN,1,856.04,2.928,23,44310,2,NaN,856.04,201,45,2,100,0,72,195517.69776,0.7
135355,1977,3,3,NaN,1,856.04,2.928,23,44310,3,NaN,814.43,301,19,2,100,0,80,195517.69776,0.7


To calculate household size computationally efficiently (i.e., to compute household size without having to iterate through nearly 8 million lines of data), we create a new variable called HHID (household ID) by assigning a unique number to each household. With HHID, we can group each household easily. 

In [ ]:
# Generate household ID
raw["HHID"] = np.nan
length = sum(raw.loc[raw['PERNUM'] == 1, 'PERNUM'])
raw.loc[raw['PERNUM'] == 1, 'HHID'] = np.arange(length)
raw = raw.fillna(method='pad')

raw.head(10)

,YEAR,SERIAL,MONTH,CPSID,ASECFLAG,ASECWTH,CPI99,STATEFIP,HHINCOME,PERNUM,CPSIDP,ASECWT,RELATE,AGE,SEX,RACE,HISPAN,EDUC,RHHINCOME,ISIZE,HHID
135351,1977,2,3,NaN,1,856.04,2.928,23,11500,1,NaN,856.04,101,48,2,100,0,110,50743.704000,1.0,0.0
135352,1977,2,3,NaN,1,856.04,2.928,23,11500,2,NaN,803.60,301,15,1,100,0,32,50743.704000,0.5,0.0
135353,1977,3,3,NaN,1,856.04,2.928,23,44310,1,NaN,856.04,101,47,1,100,0,122,195517.697760,1.0,1.0
135354,1977,3,3,NaN,1,856.04,2.928,23,44310,2,NaN,856.04,201,45,2,100,0,72,195517.697760,0.7,1.0
135355,1977,3,3,NaN,1,856.04,2.928,23,44310,3,NaN,814.43,301,19,2,100,0,80,195517.697760,0.7,1.0
135356,1977,3,3,NaN,1,856.04,2.928,23,44310,4,NaN,773.29,301,16,2,100,0,50,195517.697760,0.5,1.0
135357,1977,3,3,NaN,1,856.04,2.928,23,44310,5,NaN,846.88,301,12,2,100,0,1,195517.697760,0.5,1.0
135358,1977,4,3,NaN,1,801.57,2.928,23,23314,1,NaN,801.57,101,63,1,100,902,72,102872.931744,1.0,2.0
135359,1977,4,3,NaN,1,801.57,2.928,23,23314,2,NaN,801.57,201,63,2,100,0,72,102872.931744,0.7,2.0
135360,1977,5,3,NaN,1,657.13,2.928,23,8805,1,NaN,657.13,101,29,2,100,0,121,38852.027280,1.0,3.0


We then compute the household size for each household ID by taking the sum of all individual sizes in the household. After that, we merge HHSIZE with the raw dataset and drop PERNUM since we no longer need it. 

In [ ]:
# Generate effective size
hhsize = raw.groupby(['HHID'])['ISIZE'].agg('sum').reset_index()

# Merge effective size with raw
raw = pd.merge(raw, hhsize, on = ["HHID"])

# Rename 'sizes'
raw = raw.rename(columns={'ISIZE_x': 'ISIZE', 'ISIZE_y': 'HHSIZE'})

# Eliminate observations that has PERNUM != 1
raw = raw[raw.PERNUM == 1]

raw.head(50)

,YEAR,SERIAL,MONTH,CPSID,ASECFLAG,ASECWTH,CPI99,STATEFIP,HHINCOME,PERNUM,CPSIDP,ASECWT,RELATE,AGE,SEX,RACE,HISPAN,EDUC,RHHINCOME,ISIZE,HHID,HHSIZE
0,1977,2,3,NaN,1,856.04,2.928,23,11500,1,NaN,856.04,101,48,2,100,0,110,50743.704000,1.0,0.0,1.5
2,1977,3,3,NaN,1,856.04,2.928,23,44310,1,NaN,856.04,101,47,1,100,0,122,195517.697760,1.0,1.0,3.4
7,1977,4,3,NaN,1,801.57,2.928,23,23314,1,NaN,801.57,101,63,1,100,902,72,102872.931744,1.0,2.0,1.7
9,1977,5,3,NaN,1,657.13,2.928,23,8805,1,NaN,657.13,101,29,2,100,0,121,38852.027280,1.0,3.0,1.5
11,1977,7,3,NaN,1,632.23,2.928,23,13424,1,NaN,632.23,101,29,1,100,0,90,59233.346304,1.0,4.0,2.7
15,1977,8,3,NaN,1,681.42,2.928,23,11803,1,NaN,681.42,101,68,1,100,0,72,52080.690288,1.0,5.0,1.7
17,1977,9,3,NaN,1,886.62,2.928,23,1946,1,NaN,886.62,101,67,2,100,0,60,8586.717216,1.0,6.0,1.0
18,1977,10,3,NaN,1,867.91,2.928,23,3928,1,NaN,867.91,101,60,2,100,0,32,17332.284288,1.0,7.0,1.0
19,1977,11,3,NaN,1,816.46,2.928,23,3251,1,NaN,816.46,101,80,2,100,0,80,14345.024496,1.0,8.0,1.0
20,1977,13,3,NaN,1,854.99,2.928,23,17080,1,NaN,854.99,101,34,1,100,0,72,75365.431680,1.0,9.0,2.7


Now we can adjust RHHINCOME by HHSIZE to get *equivalized real household income*, ERHHINCOME. 

This is real household income per equivalent person.

In [ ]:
# Generate Real HHINCOME in 2018 dollars
raw["ERHHINCOME"] = np.nan
raw.ERHHINCOME = raw.RHHINCOME/raw.HHSIZE
raw.head(10)

,YEAR,SERIAL,MONTH,CPSID,ASECFLAG,ASECWTH,CPI99,STATEFIP,HHINCOME,PERNUM,CPSIDP,ASECWT,RELATE,AGE,SEX,RACE,HISPAN,EDUC,RHHINCOME,ISIZE,HHID,HHSIZE,ERHHINCOME
0,1977,2,3,NaN,1,856.04,2.928,23,11500,1,NaN,856.04,101,48,2,100,0,110,50743.704000,1.0,0.0,1.5,33829.136000
2,1977,3,3,NaN,1,856.04,2.928,23,44310,1,NaN,856.04,101,47,1,100,0,122,195517.697760,1.0,1.0,3.4,57505.205224
7,1977,4,3,NaN,1,801.57,2.928,23,23314,1,NaN,801.57,101,63,1,100,902,72,102872.931744,1.0,2.0,1.7,60513.489261
9,1977,5,3,NaN,1,657.13,2.928,23,8805,1,NaN,657.13,101,29,2,100,0,121,38852.027280,1.0,3.0,1.5,25901.351520
11,1977,7,3,NaN,1,632.23,2.928,23,13424,1,NaN,632.23,101,29,1,100,0,90,59233.346304,1.0,4.0,2.7,21938.276409
15,1977,8,3,NaN,1,681.42,2.928,23,11803,1,NaN,681.42,101,68,1,100,0,72,52080.690288,1.0,5.0,1.7,30635.700169
17,1977,9,3,NaN,1,886.62,2.928,23,1946,1,NaN,886.62,101,67,2,100,0,60,8586.717216,1.0,6.0,1.0,8586.717216
18,1977,10,3,NaN,1,867.91,2.928,23,3928,1,NaN,867.91,101,60,2,100,0,32,17332.284288,1.0,7.0,1.0,17332.284288
19,1977,11,3,NaN,1,816.46,2.928,23,3251,1,NaN,816.46,101,80,2,100,0,80,14345.024496,1.0,8.0,1.0,14345.024496
20,1977,13,3,NaN,1,854.99,2.928,23,17080,1,NaN,854.99,101,34,1,100,0,72,75365.431680,1.0,9.0,2.7,27913.122844


In [ ]:
raw.describe()

,YEAR,SERIAL,MONTH,CPSID,ASECFLAG,ASECWTH,CPI99,STATEFIP,HHINCOME,PERNUM,CPSIDP,ASECWT,RELATE,AGE,SEX,RACE,HISPAN,EDUC,RHHINCOME,ISIZE,HHID,HHSIZE,ERHHINCOME
count,2.763219e+06,2.763219e+06,2763219.0,2.053361e+06,2763219.0,2.763219e+06,2.763219e+06,2.763219e+06,2.763219e+06,2763219.0,2.053361e+06,2.763219e+06,2.763219e+06,2.763219e+06,2.763219e+06,2.763219e+06,2.763219e+06,2.763219e+06,2.763219e+06,2763219.0,2.763219e+06,2.763219e+06,2.763219e+06
mean,1.999055e+03,4.224290e+04,3.0,1.561414e+13,1.0,1.607740e+03,1.190036e+00,2.806291e+01,5.133410e+04,1.0,1.561414e+13,1.607740e+03,1.023259e+02,4.840133e+01,1.406256e+00,1.404507e+02,3.099101e+01,7.996336e+01,7.588415e+04,1.0,1.381609e+06,2.047945e+00,3.886370e+04
std,1.239818e+01,2.545720e+04,0.0,8.305102e+12,0.0,9.593081e+02,5.549597e-01,1.565348e+01,6.321888e+04,0.0,8.305102e+12,9.593081e+02,3.913122e+01,1.697419e+01,4.911336e-01,1.263333e+02,1.123604e+02,2.726908e+01,7.686676e+04,0.0,7.976728e+05,8.887119e-01,3.936242e+04
min,1.977000e+03,1.000000e+00,3.0,0.000000e+00,1.0,0.000000e+00,6.630000e-01,1.000000e+00,-3.704000e+04,1.0,0.000000e+00,0.000000e+00,1.010000e+02,0.000000e+00,1.000000e+00,1.000000e+02,0.000000e+00,1.000000e+00,-7.860026e+04,1.0,0.000000e+00,1.000000e+00,-4.386190e+04
25%,1.988000e+03,2.068400e+04,3.0,1.989021e+13,1.0,9.085700e+02,7.740000e-01,1.300000e+01,1.630300e+04,1.0,1.989021e+13,9.085700e+02,1.010000e+02,3.500000e+01,1.000000e+00,1.000000e+02,0.000000e+00,7.200000e+01,2.928208e+04,1.0,6.908045e+05,1.500000e+00,1.658307e+04
50%,2.001000e+03,4.097800e+04,3.0,1.999020e+13,1.0,1.547720e+03,9.670000e-01,2.900000e+01,3.398100e+04,1.0,1.999020e+13,1.547720e+03,1.010000e+02,4.600000e+01,1.000000e+00,1.000000e+02,0.000000e+00,7.300000e+01,5.791944e+04,1.0,1.381609e+06,1.700000e+00,2.971118e+04
75%,2.010000e+03,6.175500e+04,3.0,2.008121e+13,1.0,2.081450e+03,1.467000e+00,4.100000e+01,6.504600e+04,1.0,2.008121e+13,2.081450e+03,1.010000e+02,6.100000e+01,2.000000e+00,1.000000e+02,0.000000e+00,1.100000e+02,9.914553e+04,1.0,2.072414e+06,2.700000e+00,4.909131e+04
max,2.019000e+03,9.998600e+04,3.0,2.019031e+13,1.0,1.795753e+04,2.928000e+00,5.600000e+01,3.299997e+06,1.0,2.019031e+13,1.795753e+04,1.260000e+03,9.900000e+01,2.000000e+00,8.300000e+02,9.020000e+02,1.250000e+02,3.451328e+06,1.0,2.763218e+06,1.610000e+01,2.054233e+06


## **Section 3. Data Prep for Visualizing**

### **3.1 Preparation**

Our goal is prepare the data for rendering our final graph in JavaScript format. We generate percentiles from 5 to 95 is steps of 10 and also the median. The code below can also produce increments of 1 percentile,  but then amCharts (our visualizer) is extremely slow.

In [ ]:
# Create decile and percentile arrays
decile = np.arange(0.05, 1.05, 0.1) # 10
decile = np.insert(arr = decile, obj = 5, values = 0.5) # 11

percentile = np.arange(0.02, 1, 0.01) # 98

# Create decile and percentile name
decileName = ['5p', '15p', '25p', '35p', '45p', '50p', '55p', '65p', '75p', '85p', '95p']
percentileName = ['2p', '3p', '4p', '5p', '6p', '7p', '8p', '9p', '10p', '11p',
                  '12p', '13p', '14p', '15p', '16p', '17p', '18p', '19p', '20p',
                  '21p', '22p', '23p', '24p', '25p', '26p', '27p', '28p', '29p',
                  '30p', '31p', '32p', '33p', '34p', '35p', '36p', '37p', '38p',
                  '39p', '40p', '41p', '42p', '43p', '44p', '45p', '46p', '47p',
                  '48p', '49p', '50p', '51p', '52p', '53p', '54p', '55p', '56p',
                  '57p', '58p', '59p', '60p', '61p', '62p', '63p', '64p', '65p',
                  '66p', '67p', '68p', '69p', '70p', '71p', '72p', '73p', '74p',
                  '75p', '76p', '77p', '78p', '79p', '80p', '81p', '82p', '83p',
                  '84p', '85p', '86p', '87p', '88p', '89p', '90p', '91p', '92p',
                  '93p', '94p', '95p', '96p', '97p', '98p', '99p']

# Open HTML environment
html1_d_1year = open(in_path + 'html1_d_1year.txt', 'r')
html1_p_1year = open(in_path + 'html1_p_1year.txt', 'r')
html2 = open(in_path + 'html2.txt', 'r')

Since we allow the user to control the states being labeled, we do not label any of them. 

In [ ]:
# Labels 
label_list = ['']

We create ```state_name``` and ```state_label``` dataframe.

In [ ]:
# Create state_Name and state_label dataframes
notLabelList = list(set(state_name) - set(label_list))
notLabelDict = dict.fromkeys(notLabelList , '')
state_name = pd.DataFrame(data = state_name, index = statefip, columns = ['State'])
state_label = state_name.replace(to_replace = notLabelDict)
state_label = state_label.rename(columns={'State': 'Label'})

We create CUMWTH (cummulated household weight) and PERCENT (percentile for each household) as a way to rank income of each household

In [ ]:
# Generate cumulative weight and percentage
raw["CUMWTH"] = np.nan
raw["PERCENTH"] = np.nan

In [ ]:
raw.describe()

,YEAR,SERIAL,MONTH,CPSID,ASECFLAG,ASECWTH,CPI99,STATEFIP,HHINCOME,PERNUM,CPSIDP,ASECWT,RELATE,AGE,SEX,RACE,HISPAN,EDUC,RHHINCOME,ISIZE,HHID,HHSIZE,ERHHINCOME,CUMWTH,PERCENTH
count,2.763219e+06,2.763219e+06,2763219.0,2.053361e+06,2763219.0,2.763219e+06,2.763219e+06,2.763219e+06,2.763219e+06,2763219.0,2.053361e+06,2.763219e+06,2.763219e+06,2.763219e+06,2.763219e+06,2.763219e+06,2.763219e+06,2.763219e+06,2.763219e+06,2763219.0,2.763219e+06,2.763219e+06,2.763219e+06,0.0,0.0
mean,1.999055e+03,4.224290e+04,3.0,1.561414e+13,1.0,1.607740e+03,1.190036e+00,2.806291e+01,5.133410e+04,1.0,1.561414e+13,1.607740e+03,1.023259e+02,4.840133e+01,1.406256e+00,1.404507e+02,3.099101e+01,7.996336e+01,7.588415e+04,1.0,1.381609e+06,2.047945e+00,3.886370e+04,NaN,NaN
std,1.239818e+01,2.545720e+04,0.0,8.305102e+12,0.0,9.593081e+02,5.549597e-01,1.565348e+01,6.321888e+04,0.0,8.305102e+12,9.593081e+02,3.913122e+01,1.697419e+01,4.911336e-01,1.263333e+02,1.123604e+02,2.726908e+01,7.686676e+04,0.0,7.976728e+05,8.887119e-01,3.936242e+04,NaN,NaN
min,1.977000e+03,1.000000e+00,3.0,0.000000e+00,1.0,0.000000e+00,6.630000e-01,1.000000e+00,-3.704000e+04,1.0,0.000000e+00,0.000000e+00,1.010000e+02,0.000000e+00,1.000000e+00,1.000000e+02,0.000000e+00,1.000000e+00,-7.860026e+04,1.0,0.000000e+00,1.000000e+00,-4.386190e+04,NaN,NaN
25%,1.988000e+03,2.068400e+04,3.0,1.989021e+13,1.0,9.085700e+02,7.740000e-01,1.300000e+01,1.630300e+04,1.0,1.989021e+13,9.085700e+02,1.010000e+02,3.500000e+01,1.000000e+00,1.000000e+02,0.000000e+00,7.200000e+01,2.928208e+04,1.0,6.908045e+05,1.500000e+00,1.658307e+04,NaN,NaN
50%,2.001000e+03,4.097800e+04,3.0,1.999020e+13,1.0,1.547720e+03,9.670000e-01,2.900000e+01,3.398100e+04,1.0,1.999020e+13,1.547720e+03,1.010000e+02,4.600000e+01,1.000000e+00,1.000000e+02,0.000000e+00,7.300000e+01,5.791944e+04,1.0,1.381609e+06,1.700000e+00,2.971118e+04,NaN,NaN
75%,2.010000e+03,6.175500e+04,3.0,2.008121e+13,1.0,2.081450e+03,1.467000e+00,4.100000e+01,6.504600e+04,1.0,2.008121e+13,2.081450e+03,1.010000e+02,6.100000e+01,2.000000e+00,1.000000e+02,0.000000e+00,1.100000e+02,9.914553e+04,1.0,2.072414e+06,2.700000e+00,4.909131e+04,NaN,NaN
max,2.019000e+03,9.998600e+04,3.0,2.019031e+13,1.0,1.795753e+04,2.928000e+00,5.600000e+01,3.299997e+06,1.0,2.019031e+13,1.795753e+04,1.260000e+03,9.900000e+01,2.000000e+00,8.300000e+02,9.020000e+02,1.250000e+02,3.451328e+06,1.0,2.763218e+06,1.610000e+01,2.054233e+06,NaN,NaN


### **3.2 HHINCOME k-iles**

Code below is set for deciles, but can be changed for percentiles. Also, we need an option if we add one state per graph. 

In [ ]:
# Choose k: decile (11) or percentile (98)
k = 11

In [ ]:
if (k == 11):
  k_ile = decile
  kName = decileName

if (k == 98):
  k_ile = percentile
  kName = percentileName

A long chunk of code must run together as a whole to render all graphs for sample years from 1977 to 2019. Since it is so long, it is hard to understand. For the purpose of demostration and explanation, we walk through 1977 below to illustrate the mechanism behind the analysis. We do so by breaking the first for loop so that we render 1 graph for 1 state at a time. 

For this example, we will only work with YEAR = 1977

In [ ]:
  # Generate year dataframe -- can enter any year from 1977 to 2019
  year = raw[raw.YEAR == 1977]
  y = 1977

After that, we generate a result grid, which have the k-iles for each state at a year. 

In [ ]:
  # Generate result grid, decile-column
  result = pd.DataFrame(data = None, index = kName, columns = statefip)
  result.head(5)

,1,2,4,5,6,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,50,51,53,54,55,56
5p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


For every state, we sort the adjusted household income ascendingly. We then calculate the household cumulated weight (CUMWTH) and the household percentile (PERCENTH). For household $j$th in the RHHINCOME-sorted dataframe with $k$ households:

$$CUMWTH_j = \sum_{i=0}^{j} ASECWTH_i $$

$$PERCENTH_j = \frac{CUMWTH_j}{\sum_{i=0}^{k} ASECWTH_i}$$

To calculate RHHINCOME at each k_ile $k$ for state $s$, we select a series of RHHINCOME that has percentile under or equal to $k$ (set S):
$$S_k = \{RHHINCOME_s | PERCENTH_s \leq k \} $$

RHHINCOME at k_ile $k$ for state s is the maximum value of set $S$: 

$$ RHHINCOME_{k, s} = max(S_k) $$

In [ ]:
  # Iterate through each state
  c = 0
  for i in statefip:
    # Generate state dataframe
    state = year[year.STATEFIP == i]
    state = state.reset_index(drop = True)

    # Sort state dataframe by RHHINCOME
    state = state.sort_values('RHHINCOME')
    
    # Calculate cumulated weight and Percentage
    state.CUMWTH = state.ASECWTH.cumsum()
    state.PERCENTH = state.CUMWTH/(state.ASECWTH.sum())
    
    # Calculate decile
    r = 0
    for k in k_ile:
      result.iloc[r,c] = state.loc[state['PERCENTH'] <= k, 'RHHINCOME'].max()
      r = r + 1
    c = c + 1

The result table is presented as below:

In [ ]:
  result

,1,2,4,5,6,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,50,51,53,54,55,56
5p,8039.57,12178.5,8229.31,8458.75,12160.8,9641.3,11031.2,11635.8,8599.95,8824.99,7836.59,13237.5,9716.32,11375.4,10903.3,9791.33,8648.49,9689.84,5824.49,9173.58,10148.7,12178.5,11768.1,11340.1,6777.59,9694.25,8957.37,9420.68,11472.5,11260.7,12747.7,8255.78,10810.6,8842.64,9336.84,10832.7,8789.69,9875.17,10607.6,11406.3,8471.99,5736.24,7942.49,8877.94,10590,10757.7,9817.8,8824.99,9747.2,12708,9707.49
15p,15201,34029.2,18183.9,13237.5,20015.1,18766.3,18973.7,23580.4,17650,16741,15708.5,30005,16114.4,22062.5,21290.3,20050.4,17142.5,16330.6,11485.7,16035,23871.6,20549,22870,20465.2,13237.5,17786.8,18766.3,18585.4,22062.5,22503.7,23783.4,16255.6,18091.2,15885,17755.9,21691.8,14909.8,18585.4,18973.7,18042.7,17164.6,15002.5,13952.3,17111.7,21400.6,20518.1,21369.7,18395.7,18859,23165.6,21797.7
25p,23651,48537.5,26916.2,19891.5,30887.5,29184.2,27216.3,36712,29762.3,24533.5,25354.2,43118.9,28363.5,33535,30887.5,31770,27088.3,24860,18139.8,23787.8,37859.2,31306.7,34880.8,31686.1,18532.5,26377.9,30446.2,26475,33843.8,30410.9,35300,26475,27022.1,25045.3,26651.5,33226.1,22658.2,26607.4,30049.1,25323.3,26475,22049.2,21497.7,27366.3,31845,29881.4,31924.4,28240,26188.2,34099.8,32211.2
35p,32224.5,66187.4,38318.1,26475,40330.2,40550.8,38931.5,45669.3,39972.8,32184.7,33940.9,56453.5,40118.4,47743.2,40683.2,42201.1,38830,35741.2,27357.5,34607.2,52950,43595.5,47143.1,44142.6,25791,35719.2,42624.7,37771,44125,40639.1,45775.2,35176.4,37316.5,33226.1,35300,43436.6,33587.9,38980,39615.4,35335.3,35922.1,31703.8,28857.7,37065,42801.2,38384.3,44217.6,37709.2,35300,45682.6,42598.2
45p,42721.8,81631.2,47434.3,34417.5,51855.7,51917.4,53391.2,56479.9,46600.4,40550.8,44125,70599.9,51626.2,60504.1,50249.5,53528,50174.5,45025.1,38741.7,44566.2,66081.5,54944.4,59815.8,54794.4,35361.7,44566.2,51185,47924.1,53152.9,52067.5,57803.7,45007.5,48546.3,41662.8,44425,53250,44098.5,47893.2,51185,47941.8,46331.2,39138.8,37859.2,47659.4,52950,47253.4,56153.4,50302.5,44336.8,57874.3,52950
50p,48272.7,88964.7,52839.6,37422.4,57450.7,59573.1,59206.9,62225,49861.2,45025.1,48347.7,77823.2,55593,66946.4,55156.2,58778.9,54432.6,51017.3,44654.5,48537.5,71535.4,60702.7,66716.9,60274.7,39840.4,50567.2,56197.5,52010.1,61527.8,56250.5,63098.7,50470.1,54626.7,46353.3,51185,59339.2,46909.2,52950,55959.3,52950,50986.4,44407.4,43688.1,53656,56479.9,50549.6,61082.2,56277,49336.1,64113.6,57362.4
55p,52950,98839.9,57362.4,41106.8,64051.8,64951.9,67096.4,66628.7,54053.1,50509.8,53351.5,84534.6,61774.9,72748.8,59568.7,65746.2,60098.2,56435.8,51185,52283.7,78193.8,66319.8,72515,66191.9,45448.7,56921.2,61466.1,57362.4,67427.4,60671.8,69717.4,57362.4,61774.9,52177.8,56775.6,64237.1,53978.1,58699.4,61417.5,57472.8,56479.9,49005.2,49420,59484.9,62564.8,55156.2,65834.4,62458.9,55248.9,69743.9,62370.6
65p,63981.2,117002,68609.9,49742.1,77218.7,75453.7,81198.8,77659.9,67069.9,62127.9,68128.9,95610,72837.1,85403.9,70952.9,76781.8,70463.1,66822.8,63879.7,62370.6,92106.4,76195,84719.9,79424.9,55054.7,70599.9,70092.5,70427.8,77659.9,73159.2,82822.5,68680.5,74571.2,62088.2,68618.7,75767,64863.7,70732.3,70820.6,67228.8,67775.9,59264.2,61532.3,72695.9,73423.9,64744.6,78727.8,75365.4,67511.2,79464.6,73816.6
75p,77262.8,139589,82513.7,62414.8,92212.3,90037,93986.2,91413.7,83378.5,73781.3,81017.8,109620,83285.9,100164,83837.4,88832.4,85077.3,79733.8,75436,73688.7,110083,90619.4,100393,97092.6,66187.4,86043.7,84075.7,83502.1,92146.2,87190.9,100362,83837.4,89132.4,76834.8,82248.9,88249.9,76336.2,85575.9,84058,77558.4,83850.7,75899.3,75021.3,87243.9,87367.4,79424.9,96316,89917.8,80431,92635.9,88249.9
85p,94859.8,174735,98853.1,76243.5,114165,110259,114879,111839,113935,90385.6,100164,132256,100605,123647,104197,110180,106284,98133.9,97074.9,90456.2,133257,112227,119945,118228,79424.9,104488,99210.6,97869.2,114420,110167,120629,103844,111023,92737.4,99766.5,106712,97224.

We transpose the result dataframe so that we can merge them with codeFirst_oneYear and codeThird_oneYear

In [ ]:
  # Transpose result table: column-decile
  result = result.T
  result.head(10)

,5p,15p,25p,35p,45p,50p,55p,65p,75p,85p,95p
1,8039.57,15201,23651,32224.5,42721.8,48272.7,52950,63981.2,77262.8,94859.8,131934
2,12178.5,34029.2,48537.5,66187.4,81631.2,88964.7,98839.9,117002,139589,174735,223122
4,8229.31,18183.9,26916.2,38318.1,47434.3,52839.6,57362.4,68609.9,82513.7,98853.1,152897
5,8458.75,13237.5,19891.5,26475,34417.5,37422.4,41106.8,49742.1,62414.8,76243.5,115519
6,12160.8,20015.1,30887.5,40330.2,51855.7,57450.7,64051.8,77218.7,92212.3,114165,166192
8,9641.3,18766.3,29184.2,40550.8,51917.4,59573.1,64951.9,75453.7,90037,110259,144178
9,11031.2,18973.7,27216.3,38931.5,53391.2,59206.9,67096.4,81198.8,93986.2,114879,163527
10,11635.8,23580.4,36712,45669.3,56479.9,62225,66628.7,77659.9,91413.7,111839,144496
11,8599.95,17650,29762.3,39972.8,46600.4,49861.2,54053.1,67069.9,83378.5,113935,172087
12,8824.99,16741,24533.5,32184.7,40550.8,45025.1,50509.8,62127.9,73781.3,90385.6,135464


We sort the result by median (50p) and concatenate it with ```state_name```, ```state_label```, and ```colors```. 

In [ ]:
  # Sort the result by median
  result = result.sort_values(by = ['50p'], ascending = True)
  colors1977 = pd.DataFrame(data = list(colors.Color), index = result.index, columns=['Color'])
  result = pd.concat([state_name, result, state_label, colors1977], axis = 1)
  result.head(5)

,State,5p,15p,25p,35p,45p,50p,55p,65p,75p,85p,95p,Label,Color
1,Alabama,8039.57,15201,23651,32224.5,42721.8,48272.7,52950,63981.2,77262.8,94859.8,131934,,#FF5100
2,Alaska,12178.5,34029.2,48537.5,66187.4,81631.2,88964.7,98839.9,117002,139589,174735,223122,,#00FF00
4,Arizona,8229.31,18183.9,26916.2,38318.1,47434.3,52839.6,57362.4,68609.9,82513.7,98853.1,152897,,#FFCC00
5,Arkansas,8458.75,13237.5,19891.5,26475,34417.5,37422.4,41106.8,49742.1,62414.8,76243.5,115519,,#FF0000
6,California,12160.8,20015.1,30887.5,40330.2,51855.7,57450.7,64051.8,77218.7,92212.3,114165,166192,,#A3FF00


In [ ]:
result

,State,5p,15p,25p,35p,45p,50p,55p,65p,75p,85p,95p,Label,Color
1,Alabama,8039.57,15201,23651,32224.5,42721.8,48272.7,52950,63981.2,77262.8,94859.8,131934,,#FF5100
2,Alaska,12178.5,34029.2,48537.5,66187.4,81631.2,88964.7,98839.9,117002,139589,174735,223122,,#00FF00
4,Arizona,8229.31,18183.9,26916.2,38318.1,47434.3,52839.6,57362.4,68609.9,82513.7,98853.1,152897,,#FFCC00
5,Arkansas,8458.75,13237.5,19891.5,26475,34417.5,37422.4,41106.8,49742.1,62414.8,76243.5,115519,,#FF0000
6,California,12160.8,20015.1,30887.5,40330.2,51855.7,57450.7,64051.8,77218.7,92212.3,114165,166192,,#A3FF00
8,Colorado,9641.3,18766.3,29184.2,40550.8,51917.4,59573.1,64951.9,75453.7,90037,110259,144178,,#7AFF00
9,Connecticut,11031.2,18973.7,27216.3,38931.5,53391.2,59206.9,67096.4,81198.8,93986.2,114879,163527,,#8EFF00
10,Delaware,11635.8,23580.4,36712,45669.3,56479.9,62225,66628.7,77659.9,91413.7,111839,144496,,#47FF00
11,District of Columbia,8599.95,17650,29762.3,39972.8,46600.4,49861.2,54053.1,67069.9,83378.5,113935,172087,,#FF7A00
12,Florida,8824.99,16741,24533.5,32184.7,40550.8,45025.1,50509.8,62127.9,73781.3,90385.6,135464,,#FF3300


### 3.3 Population

We compute an estimate of the number of households in the population (POP) and then normalize it for each state in a new dataframe (NORMPOP). 

For state $s$ that has $k$ households in year $y$, we could make each state's population relative to the smallest state, like this:

$$POP_s = \sum_{i=0}^{k} ASECWTH_i$$

$$NORMPOP_s = \frac{POP_s}{min(POP_s)}$$ 



In [ ]:
  # Compute state population and normalized state population
  pop = pd.DataFrame(index = statefip)
  pop["POP"] = np.nan
  pop.POP = year.groupby(['STATEFIP'])['ASECWT'].agg('sum')
  pop["NORMPOP"] = np.nan
  pop.NORMPOP = round(pop.POP/(pop.POP.min()))
  pop

,POP,NORMPOP
1,1301713.96,11.0
2,116622.28,1.0
4,789233.03,7.0
5,687999.56,6.0
6,7934076.94,68.0
8,1046900.30,9.0
9,1078354.27,9.0
10,188215.87,2.0
11,252873.12,2.0
12,3079626.13,26.0


However, this turns out to produce an ugly graph (with extremely thin slices for the smallest states) and takes a long time to render in a browser.

To improve visibility and rendering, we normalize to the 10th percentile which makes about 1/4 of the states have a width of 1 in the graph. California (the most populous state) has a width 34 times the smallest states (in actuality it should be 68 times).

In [ ]:
pop.NORMPOP = round(pop.POP/(np.percentile(pop.POP,10)))
pop

,POP,NORMPOP
1,1301713.96,6.0
2,116622.28,1.0
4,789233.03,3.0
5,687999.56,3.0
6,7934076.94,34.0
8,1046900.30,5.0
9,1078354.27,5.0
10,188215.87,1.0
11,252873.12,1.0
12,3079626.13,13.0


In [ ]:
  # Replicate each state's dataline with its respective replication number
  for i in statefip:
    rep = pop.loc[i,'NORMPOP'] - 1
    rep = int(rep)
    line = pd.DataFrame(result.loc[i]).T
    line.loc[i, 'Label'] = ''
    for i in range(0,rep): result = pd.concat([result, line])
  result.head(50)

,State,5p,15p,25p,35p,45p,50p,55p,65p,75p,85p,95p,Label,Color
1,Alabama,8039.57,15201,23651,32224.5,42721.8,48272.7,52950,63981.2,77262.8,94859.8,131934,,#FF5100
2,Alaska,12178.5,34029.2,48537.5,66187.4,81631.2,88964.7,98839.9,117002,139589,174735,223122,,#00FF00
4,Arizona,8229.31,18183.9,26916.2,38318.1,47434.3,52839.6,57362.4,68609.9,82513.7,98853.1,152897,,#FFCC00
5,Arkansas,8458.75,13237.5,19891.5,26475,34417.5,37422.4,41106.8,49742.1,62414.8,76243.5,115519,,#FF0000
6,California,12160.8,20015.1,30887.5,40330.2,51855.7,57450.7,64051.8,77218.7,92212.3,114165,166192,,#A3FF00
8,Colorado,9641.3,18766.3,29184.2,40550.8,51917.4,59573.1,64951.9,75453.7,90037,110259,144178,,#7AFF00
9,Connecticut,11031.2,18973.7,27216.3,38931.5,53391.2,59206.9,67096.4,81198.8,93986.2,114879,163527,,#8EFF00
10,Delaware,11635.8,23580.4,36712,45669.3,56479.9,62225,66628.7,77659.9,91413.7,111839,144496,,#47FF00
11,District of Columbia,8599.95,17650,29762.3,39972.8,46600.4,49861.2,54053.1,67069.9,83378.5,113935,172087,,#FF7A00
12,Florida,8824.99,16741,24533.5,32184.7,40550.8,45025.1,50509.8,62127.9,73781.3,90385.6,135464,,#FF3300


Sort the result dataframe by the median to get the 3D visualization to plot the states in ascending order of household income. 

In [ ]:
  # Sort the result by median
  result = result.sort_values(by = ['50p', 'State'], ascending = True)
  result

,State,5p,15p,25p,35p,45p,50p,55p,65p,75p,85p,95p,Label,Color
5,Arkansas,8458.75,13237.5,19891.5,26475,34417.5,37422.4,41106.8,49742.1,62414.8,76243.5,115519,,#FF0000
5,Arkansas,8458.75,13237.5,19891.5,26475,34417.5,37422.4,41106.8,49742.1,62414.8,76243.5,115519,,#FF0000
5,Arkansas,8458.75,13237.5,19891.5,26475,34417.5,37422.4,41106.8,49742.1,62414.8,76243.5,115519,,#FF0000
28,Mississippi,6777.59,13237.5,18532.5,25791,35361.7,39840.4,45448.7,55054.7,66187.4,79424.9,114138,,#FF0A00
28,Mississippi,6777.59,13237.5,18532.5,25791,35361.7,39840.4,45448.7,55054.7,66187.4,79424.9,114138,,#FF0A00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,Maryland,10148.7,23871.6,37859.2,52950,66081.5,71535.4,78193.8,92106.4,110083,133257,191238,,#14FF00
24,Maryland,10148.7,23871.6,37859.2,52950,66081.5,71535.4,78193.8,92106.4,110083,133257,191238,,#14FF00
24,Maryland,10148.7,23871.6,37859.2,52950,66081.5,71535.4,78193.8,92106.4,110083,133257,191238,,#14FF00
15,Hawaii,13237.5,30005,43118.9,56453.5,70599.9,77823.2,84534.6,95610,109620,132256,172414,,#0AFF00


In [ ]:
result.reset_index(drop = True, inplace = True)
result

,State,5p,15p,25p,35p,45p,50p,55p,65p,75p,85p,95p,Label,Color
0,Arkansas,8458.75,13237.5,19891.5,26475,34417.5,37422.4,41106.8,49742.1,62414.8,76243.5,115519,,#FF0000
1,Arkansas,8458.75,13237.5,19891.5,26475,34417.5,37422.4,41106.8,49742.1,62414.8,76243.5,115519,,#FF0000
2,Arkansas,8458.75,13237.5,19891.5,26475,34417.5,37422.4,41106.8,49742.1,62414.8,76243.5,115519,,#FF0000
3,Mississippi,6777.59,13237.5,18532.5,25791,35361.7,39840.4,45448.7,55054.7,66187.4,79424.9,114138,,#FF0A00
4,Mississippi,6777.59,13237.5,18532.5,25791,35361.7,39840.4,45448.7,55054.7,66187.4,79424.9,114138,,#FF0A00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,Maryland,10148.7,23871.6,37859.2,52950,66081.5,71535.4,78193.8,92106.4,110083,133257,191238,,#14FF00
322,Maryland,10148.7,23871.6,37859.2,52950,66081.5,71535.4,78193.8,92106.4,110083,133257,191238,,#14FF00
323,Maryland,10148.7,23871.6,37859.2,52950,66081.5,71535.4,78193.8,92106.4,110083,133257,191238,,#14FF00
324,Hawaii,13237.5,30005,43118.9,56453.5,70599.9,77823.2,84534.6,95610,109620,132256,172414,,#0AFF00


The code below is used when we render the graph in amCharts. It enables the user to add a label to the middle slice in the graph. 

In [ ]:
result["Middle"] = np.nan
counter = 0
for state in result.State.drop_duplicates():
  temp = result[result.State == state]
  temp_size = len(temp.index)
  middle = (temp_size // 2)
  counter = counter + middle
  result.loc[counter, 'Middle'] = 1
  counter = counter - middle + temp_size

In [ ]:
pd.set_option('display.max_rows', 400)
result.head(400)

,State,5p,15p,25p,35p,45p,50p,55p,65p,75p,85p,95p,Label,Color,Middle
0,Arkansas,8458.75,13237.5,19891.5,26475,34417.5,37422.4,41106.8,49742.1,62414.8,76243.5,115519,,#FF0000,NaN
1,Arkansas,8458.75,13237.5,19891.5,26475,34417.5,37422.4,41106.8,49742.1,62414.8,76243.5,115519,,#FF0000,1.0
2,Arkansas,8458.75,13237.5,19891.5,26475,34417.5,37422.4,41106.8,49742.1,62414.8,76243.5,115519,,#FF0000,NaN
3,Mississippi,6777.59,13237.5,18532.5,25791,35361.7,39840.4,45448.7,55054.7,66187.4,79424.9,114138,,#FF0A00,NaN
4,Mississippi,6777.59,13237.5,18532.5,25791,35361.7,39840.4,45448.7,55054.7,66187.4,79424.9,114138,,#FF0A00,NaN
5,Mississippi,6777.59,13237.5,18532.5,25791,35361.7,39840.4,45448.7,55054.7,66187.4,79424.9,114138,,#FF0A00,1.0
6,Mississippi,6777.59,13237.5,18532.5,25791,35361.7,39840.4,45448.7,55054.7,66187.4,79424.9,114138,,#FF0A00,NaN
7,Tennessee,7942.49,13952.3,21497.7,28857.7,37859.2,43688.1,49420,61532.3,75021.3,91691.7,127720,,#FF1400,NaN
8,Tennessee,7942.49,13952.3,21497.7,28857.7,37859.2,43688.1,49420,61532.3,75021.3,91691.7,127720,,#FF1400,NaN
9,Tennessee,7942.49,13952.3,21497.7,28857.7,37859.2,43688.1,49420,61532.3,75021.3,91691.7,127720,,#FF1400,NaN


## **Section 4: Output to JSON**

We convert the result dataframe to JSON format.

In [ ]:
  # Convert dataframe to JSON
  result = result.to_json(orient = 'records')
  result = json.loads(result, object_pairs_hook = OrderedDict)

  # Make JSON format readable
  result = json.dumps(result, indent = 4, sort_keys = False)

Since we broke a for-loop into several cells (for debugging purpose), we set k again, just in case: 

In [ ]:
  # Choose k: decile (11) or percentile (98)
  k = 11

In [ ]:
  # Insert result to HTML environment
  if k == 11: html1 = open(in_path + 'html1_d_1year.txt', 'r')
  if k == 98: html1 = open(in_path + 'html1_p_1year.txt', 'r')
  html2 = open(in_path + 'html2.txt', 'r')

Finally, we output the data in JSON format so amCharts can create the graph.

In [ ]:
  # Insert result to HTML environment
  with open(out_path + 'year_d/' + str(y) + '_d.html', 'w') as outfile:
    outfile.write(html1.read())
    outfile.write(result)
    outfile.write(html2.read())

After the demonstration for 1977, we now run the code for the full data for samples from 1977 to 2019, which yields HHINCOME from 1976 to 2018. Note that Python's range function does not include the last (stop) value so to get from 1977 to 2019, we use range(1977, 2020).

In [ ]:
for y in range(1977, 2020):
  # Generate result grid, decile-column
  result = pd.DataFrame(data = None, index = kName, columns = statefip)

  # Generate year dataframe
  year = raw[raw.YEAR == y]

  # Iterate through each state
  c = 0
  for i in statefip:
    # Generate state dataframe
    state = year[year.STATEFIP == i]
    state = state.reset_index(drop = True)

    # Sort state dataframe by RHHINCOME
    state = state.sort_values('RHHINCOME')
    
    # Calculate cumulated weight and Percentage
    state.CUMWTH = state.ASECWTH.cumsum()
    state.PERCENTH = state.CUMWTH/(state.ASECWTH.sum())

    # Calculate decile
    r = 0
    for k in k_ile:
      result.iloc[r,c] = state.loc[state['PERCENTH'] <= k, 'RHHINCOME'].max()
      r = r + 1
    c = c + 1

  # Transpose result table: column-decile
  result = result.T
  result.head(10)

  # Sort the result by median
  result = result.sort_values(by = ['50p'], ascending = True)
  # We won't regenerate color gradient, as we always use color gradient of 1977. 
  # colors1977 = pd.DataFrame(data = list(colors.Color), index = result.index, columns=['Color'])
  result = pd.concat([state_name, result, state_label, colors1977], axis = 1)
  result.head(5)

  # Compute state population and normalized state population
  pop = pd.DataFrame(index = statefip)
  pop["POP"] = np.nan
  pop.POP = year.groupby(['STATEFIP'])['ASECWT'].agg('sum')

  pop["NORMPOP"] = np.nan
  pop.NORMPOP = round(pop.POP/(np.percentile(pop.POP,10)))

  # Replicate each state's dataline with its respective replication number
  for i in statefip:
    rep = pop.loc[i,'NORMPOP'] - 1
    rep = int(rep)
    line = pd.DataFrame(result.loc[i]).T
    line.loc[i, 'Label'] = ''
    for i in range(0,rep): result = pd.concat([result, line])
  result.head(10)  

  # Sort the result by median
  result = result.sort_values(by = ['50p', 'State'], ascending = True)

  # Add the middle property
  result.reset_index(drop = True, inplace = True)

  result["Middle"] = np.nan
  counter = 0
  for state in result.State.drop_duplicates():
    temp = result[result.State == state]
    temp_size = len(temp.index)
    middle = (temp_size // 2)
    counter = counter + middle
    result.loc[counter, 'Middle'] = 1
    counter = counter - middle + temp_size

  # Convert dataframe to JSON
  result = result.to_json(orient = 'records')
  result = json.loads(result, object_pairs_hook = OrderedDict)

  # Make JSON format readable
  result = json.dumps(result, indent = 4, sort_keys = False)

  # Insert result to HTML environment
  if k == 11: html1 = open(in_path + 'html1_d_1year.txt', 'r')
  if k == 98: html1 = open(in_path + 'html1_p_1year.txt', 'r')
  html2 = open(in_path + 'html2.txt', 'r')

  # Save JSON file -- y-1 adjusts sample year to HHINCOME year
  with open(out_path + 'year_d_js/' + str(y-1) + '_d_RHH.js', 'w') as outfile:
    outfile.write("var data =")
    outfile.write(result)

Run the code for ERHHINCOME.

In [ ]:
for y in range(1977, 2020):
  # Generate result grid, decile-column
  result = pd.DataFrame(data = None, index = kName, columns = statefip)

  # Generate year dataframe
  year = raw[raw.YEAR == y]

  # Iterate through each state
  c = 0
  for i in statefip:
    # Generate state dataframe
    state = year[year.STATEFIP == i]
    state = state.reset_index(drop = True)

    # Sort state dataframe by ERHHINCOME
    state = state.sort_values('ERHHINCOME')
    
    # Calculate cumulated weight and Percentage
    state.CUMWTH = state.ASECWTH.cumsum()
    state.PERCENTH = state.CUMWTH/(state.ASECWTH.sum())

    # Calculate decile
    r = 0
    for k in k_ile:
      result.iloc[r,c] = state.loc[state['PERCENTH'] <= k, 'ERHHINCOME'].max()
      r = r + 1
    c = c + 1

  # Transpose result table: column-decile
  result = result.T
  result.head(10)

  # Sort the result by median
  result = result.sort_values(by = ['50p'], ascending = True)
  # We won't regenerate color gradient, as we always use color gradient of 1977. 
  # colors1977 = pd.DataFrame(data = list(colors.Color), index = result.index, columns=['Color'])
  result = pd.concat([state_name, result, state_label, colors1977], axis = 1)
  result.head(5)

  # Compute state population and normalized state population
  pop = pd.DataFrame(index = statefip)
  pop["POP"] = np.nan
  pop.POP = year.groupby(['STATEFIP'])['ASECWT'].agg('sum')

  pop["NORMPOP"] = np.nan
  pop.NORMPOP = round(pop.POP/(np.percentile(pop.POP,10)))

  # Replicate each state's dataline with its respective replication number
  for i in statefip:
    rep = pop.loc[i,'NORMPOP'] - 1
    rep = int(rep)
    line = pd.DataFrame(result.loc[i]).T
    line.loc[i, 'Label'] = ''
    for i in range(0,rep): result = pd.concat([result, line])
  result.head(10)  

  # Sort the result by median
  result = result.sort_values(by = ['50p', 'State'], ascending = True)

  # Add the middle property
  result.reset_index(drop = True, inplace = True)
  result["Middle"] = np.nan
  counter = 0
  for state in result.State.drop_duplicates():
    temp = result[result.State == state]
    temp_size = len(temp.index)
    middle = (temp_size // 2)
    counter = counter + middle
    result.loc[counter, 'Middle'] = 1
    counter = counter - middle + temp_size

  # Convert dataframe to JSON
  result = result.to_json(orient = 'records')
  result = json.loads(result, object_pairs_hook = OrderedDict)

  # Make JSON format readable
  result = json.dumps(result, indent = 4, sort_keys = False)

  # Insert result to HTML environment
  if k == 11: html1 = open(in_path + 'html1_d_1year.txt', 'r')
  if k == 98: html1 = open(in_path + 'html1_p_1year.txt', 'r')
  html2 = open(in_path + 'html2.txt', 'r')

  # Save JSON file -- y-1 adjusts sample year to HHINCOME year
  with open(out_path + 'year_d_js/' + str(y-1) + '_d_ERHH.js', 'w') as outfile:
    outfile.write("var data =")
    outfile.write(result)

This concludes the Python processing part of the project. With the data prepared, we turn to amCharts to render the graph.

In [ ]:
# import pandas as pd
# import json
# from collections import OrderedDict
# from google.colab import drive
# drive.mount('/content/gdrive')
# in_path = 'gdrive/My Drive/Colab Notebooks/code/'
# out_path = 'gdrive/My Drive/Colab Notebooks/oneState/state_d/'

# # Import raw data state codes, color codes, and deflator
# raw = pd.read_csv(in_path + "raw.csv")
# codeFirst_oneState = pd.read_csv(in_path + "codeFirst_oneState.csv", index_col = 0)
# codeThird_oneState = pd.read_csv(in_path + "codeThird_oneState.csv", index_col = 0)
# deflator = pd.read_csv(in_path +"COLI.csv")

# # Select data with HFLAG != 1 and then drop HFLAG
# raw = raw[raw.HFLAG != 1]
# raw = raw.drop(columns = ['HFLAG'])

# # Generate constants
# stateList = [1,2,4,5,6,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,
#              28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,
#              50,51,53,54,55,56]

# # stateList = [12]

# decile = [0.05,0.15,0.25,0.35,0.45,0.50,0.55,0.65,0.75,0.85,0.95]
# value = ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
#          '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
#          '', '', '', '', '']
# yearList = [1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988,
#             1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
#             2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
#             2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

# # Iterate through each state
# for s in stateList:
  
#   # Generate result grid, decile-column
#   result = pd.DataFrame(columns = yearList)
#   for i in range(0, 11): result.loc[i] = value
  
#   # Generate state dataframe
#   state = raw[raw.STATEFIP == s]

#   # Eliminate observations that has PERNUM != 1
#   person = state[state.PERNUM == 1]
  
#   # Merge 2 file: for raw, every row that has STATEFIP and YEAR match with 
#   # that row in deflator will get the same deflator value.
#   person = pd.merge(person, deflator, on = ["YEAR","STATEFIP"])
  
#   # Rearrange culumns order
#   person = person[['YEAR', 'ASECWTH', 'STATEFIP', 'HHINCOME', 'PERNUM', 
#                    'ASECWT','DEFLATOR','SEX','RACE','HISPAN','EDUC']]
  
#   # Generate deflated household income column
#   person.insert(7, 'DHHINCOME', '')
  
#   # Iterate through the entire 'person' to calculate deflated income
#   for i in range (0, len(person)):
#     person.iloc[i, 7] = person.iloc[i, 3]/person.iloc[i, 6]
  
#   # Sort the remainded observation by YEAR
#   person.sort_values('YEAR')
  
#   # Iterate through each year
#   c = 0
#   for i in yearList:
#     # Generate year dataframe
#     year = person[person.YEAR == i]
    
#     # Sort year dataframe by HHINCOME
#     year = year.sort_values('HHINCOME')
    
#     # Calculate cumulated weight and Percentage
#     year.insert(8, 'CUMWT', '')
#     year.insert(9, 'PERCENT', '')
#     year.iloc[0, 8] = year.iloc[0, 1]
#     year.iloc[0, 9] = year.iloc[0, 8]/(year.sum().ASECWT)
#     for i in range (1, len(year)):
#       year.iloc[i, 8] = year.iloc[i-1, 8] + year.iloc[i, 1]
#       year.iloc[i, 9] = year.iloc[i, 8]/(year.sum().ASECWT)
    
#     # Calculate decile
#     r = 0
#     for d in decile:
#       for i in range (0, len(year)):
#         if (d < year.iloc[i, 9]):
#           result.iloc[r,c] = year.iloc[i, 7]
#           r = r + 1
#           break
#     c = c + 1
    
#   # Transpose result table: column-decile
#   result = result.transpose()

#   # Type casting result.index (type casting YEAR) to integer
#   result.index = result.index.map(int)

#   # Merge state dataframe with code dataframe
#   result = pd.merge(codeFirst_oneState, result, left_index = True, right_index = True)
#   result = pd.merge(result, codeThird_oneState, left_index = True, right_index = True)

#   result.insert(14, 'POP','')
#   r = 0
#   for i in yearList:
#     year = state[state.YEAR == i]
#     result.iloc[r, 14] = year.sum().ASECWT
#     r = r + 1

#   result.insert(15, 'NORMPOP', '')
#   for i in range(0, len(result)):
#     result.iloc[i, 15] = round(10*(result.iloc[i, 14])/(result['POP'].min()))
# #     result.iloc[i, 15] = result.iloc[i, 14]/(result['POP'].min())
#   for i in yearList:
#     rep = result.loc[int(i),'NORMPOP'] - 1
#     rep = int(rep)
#     # The following statement need .copy() at the end for explicit reason
#     # More information: https://www.dataquest.io/blog/settingwithcopywarning/
#     line = result[result.index == i].copy()
#     line.name = i
#     # Remove the name of the state (so that the name does not repeat too many time)
#     line.iloc[0, 12] = ''
#     for i in range(0,rep): result = result.append(line)
        
# #   # result.to_csv(out_path+str(y)+'withPop.csv')
#   result = result.drop(columns = ['POP', 'NORMPOP'])
      
#   # Sort the result by year
#   result = result.sort_values("Year", ascending = True)

#   # Rename index column and role
#   result = result.rename(index = str, columns = {0: "5p", 1: "15p", 2: "25p",
#                                                  3: "35p", 4: "45p", 5: "50p",
#                                                  6: "55p", 7: "65p", 8: "75p",
#                                                  9: "85p", 10: "95p"})
  
#   result.to_csv(out_path+str(s) + '_d.csv')

#   # Convert dataframe to JSON
#   result = result.to_json(orient = 'records')
#   result = json.loads(result, object_pairs_hook = OrderedDict)

#   # Make JSON format readable
#   result = json.dumps(result, indent = 4, sort_keys = False)

#   # Save JSON to text format
#   with open(out_path + str(s) + '_d.txt', 'w') as f:
#     f.writelines(result)

#   # Glue data with html environment
#   filenames = [in_path + 'first_d_oneState.txt', out_path + str(s) + '_d.txt',
#                in_path + 'third.txt']
#   with open(out_path + str(s) + '_d.html', 'w') as outfile:
#     for i in filenames:
#       with open (i) as infile:
#         outfile.write(infile.read())

References:

*   Color generator: https://www.strangeplanet.fr/work/gradient-generator/index.php

*   AmChart documentation:  https://docs.amcharts.com/3/javascriptcharts/AmGraph

*   Jack Blundell's graph: https://jackblun.github.io/Globalinc/html/fig_1980.html